In [1]:
import requests
import json
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2 
import io
from config import AWS

In [7]:
#connecting to postgres DB in AWS RDS to create a dataFrame
#engine = create_engine(AWS)
#metro_df = pd.read_sql('bls50metro2020', engine)

#importing clean CSV files into variable.  Had to rename the file because of parsing error
metro_file = "..\data_files\metro_2020_clean.csv"
tech_file = "..\data_files\TechnicalJobs_DF_draft.csv"
demo_city_file = "..\data_files\metro_demographic_by_city_state.csv"
demo_zipcode_file = "..\data_files\metro_demographic_by_zipcode.csv"

#using clean data to create a dataframe for each DB table
metro_df = pd.read_csv(metro_file)
tech_metro_df = pd.read_csv(tech_file)
demo_city_df = pd.read_csv(demo_city_file)
demo_zip_df = pd.read_csv(demo_zipcode_file)


In [9]:
metro_df.head()

,Unnamed: 0,area,area_title,prim_state,occ_code,occ_title,o_group,tot_emp,emp_prse,jobs_1000,...,h_pct10,h_pct25,h_median,h_pct75,h_pct90,a_pct10,a_pct25,a_median,a_pct75,a_pct90
0,0,10180,"Abilene, TX",TX,00-0000,All Occupations,total,66060.0,1.9,1000.000,...,9.13,11.40,16.41,24.82,36.23,18990.0,23710.0,34130.0,51620.0,75370.0
1,1,10180,"Abilene, TX",TX,11-0000,Management Occupations,major,2910.0,4.5,44.068,...,17.38,26.74,36.37,51.68,73.88,36150.0,55620.0,75640.0,107500.0,153670.0
2,2,10180,"Abilene, TX",TX,11-1021,General and Operations Managers,detailed,1320.0,7.4,20.036,...,12.67,24.02,32.96,50.11,72.93,26350.0,49960.0,68550.0,104230.0,151700.0
3,4,10180,"Abilene, TX",TX,11-2030,Public Relations and Fundraising Managers,detailed,40.0,31.5,0.549,...,28.71,32.78,36.24,41.23,83.81,59710.0,68180.0,75380.0,85760.0,174320.0
4,5,10180,"Abilene, TX",TX,11-3010,Administrative Services and Facilities Managers,detailed,140.0,12.3,2.045,...,24.93,28.28,35.06,45.45,55.25,51850.0,58820.0,72920.0,94530.0,114920.0


In [10]:
#using the jobs_df and only capturing data for ML
jobs_df = metro_df[['area', 'area_title', 'prim_state' , 'occ_code','occ_title', 'tot_emp', 'emp_prse' , 'jobs_1000']]
city_df = metro_df[['area', 'area_title','prim_state','occ_title',]]

In [12]:
city_df.head()

,area,area_title,prim_state,occ_title
0,10180,"Abilene, TX",TX,All Occupations
1,10180,"Abilene, TX",TX,Management Occupations
2,10180,"Abilene, TX",TX,General and Operations Managers
3,10180,"Abilene, TX",TX,Public Relations and Fundraising Managers
4,10180,"Abilene, TX",TX,Administrative Services and Facilities Managers


In [13]:
jobs_df.head()

,area,area_title,prim_state,occ_code,occ_title,tot_emp,emp_prse,jobs_1000
0,10180,"Abilene, TX",TX,00-0000,All Occupations,66060.0,1.9,1000.000
1,10180,"Abilene, TX",TX,11-0000,Management Occupations,2910.0,4.5,44.068
2,10180,"Abilene, TX",TX,11-1021,General and Operations Managers,1320.0,7.4,20.036
3,10180,"Abilene, TX",TX,11-2030,Public Relations and Fundraising Managers,40.0,31.5,0.549
4,10180,"Abilene, TX",TX,11-3010,Administrative Services and Facilities Managers,140.0,12.3,2.045


In [16]:
ml_df = jobs_df.drop(['area_title','prim_state','occ_title'], axis=1)
ml_df['occ_code'] = jobs_df['occ_code'].astype(str).apply(lambda x: x.replace('-', '')).astype(int)
ml_df.head()

,area,occ_code,tot_emp,emp_prse,jobs_1000
0,10180,0,66060.0,1.9,1000.000
1,10180,110000,2910.0,4.5,44.068
2,10180,111021,1320.0,7.4,20.036
3,10180,112030,40.0,31.5,0.549
4,10180,113010,140.0,12.3,2.045


In [19]:
demo_city_df.head(10)

,Unnamed: 0,city,state,area_name,population,median_age,average_income,family_poverty,educational_attainment_bachelors,educational_attainment_graduate,...,educational_attainment_no_diploma,educational_attainment_some_college,race_asian,race_white,race_black,race_hispanic,race_native,race_islander,race_two,race_other
0,0,Abilene,Texas,"Abilene, TX",85075,27,49583,9368,12687,6935,...,10738,29001,2253,53385,8597,18241,524,15,1860,200
1,1,Akron,Ohio,"Akron, OH",94894,33,42943,25508,10769,5611,...,13742,31013,2164,54490,31493,1761,134,22,4765,65
2,2,Albany,Georgia,"Albany, GA",64884,37,59234,13830,9365,7235,...,9037,21217,948,20685,40100,1737,96,24,1241,53
3,3,Albuquerque,New Mexico,"Albuquerque, NM",319991,35,61589,42306,66219,56998,...,25103,105387,8437,153880,9545,126499,13606,213,6795,1016
4,4,Alexandria,Louisiana,"Alexandria, LA",59184,38,57892,10955,7768,5315,...,9059,16030,1856,23883,30783,1494,206,7,924,31
5,5,Amarillo,Texas,"Amarillo, TX",196907,35,67009,26741,27313,13009,...,35426,67945,7044,109181,13680,61803,643,145,4232,179
6,6,Ames,Iowa,"Ames, IA",69189,22,35781,2342,19565,19574,...,1540,13890,6813,55861,1683,2352,175,34,2257,14
7,7,Anchorage,Alaska,"Anchorage, AK",867081,33,102234,47730,190701,109314,...,57651,305823,80919,507582,46935,77619,60072,20823,71037,2094
8,8,Ann Arbor,Michigan,"Ann Arbor, MI",129170,29,74733,5607,38922,53784,...,3802,22292,15555,94161,8860,4852,231,53,5265,193
9,9,Asheville,North Carolina,"Asheville, NC",44784,44,65450,3655,9949,5578,...,4414,13133,106,39421,1611,2914,85,49,579,19


In [ ]:
#connecting to postgres DB to create a new table in AWS RDS called
engine = create_engine(AWS)

metro_df.to_sql('bls_metro2020', engine, if_exists='replace')
tech_metro_df.to_sql('tech_jobs2020', engine, if_exists='replace')
demo_city_df.to_sql('demo_city2020' engine, if_exists='replace')
demo_zip_df.to_sql('demo_zip2020', engine, if_exists='replace')
city_df.to_sql('city2020', engine, if_exists='replace')
jobs_df.to_sql('jobs2020', engine, if_exists='replace')
ml_df.to_sql('ml2020', engine, if_exists='replace')